In [ ]:
from flask import Flask, request, redirect, jsonify
import oracledb

app = Flask(__name__)

conexion = oracledb.connect(
    user="banquetes_admin",
    password="banquetes2025",
    dsn="localhost/XEPDB1"
)
cursor = conexion.cursor()

@app.route("/registrarUsuario", methods=["POST"])
def registrar_usuario():
    try:
        data = request.get_json(force=True)

        usuario = data["usuario"]
        password = data["password"]
        email = data["email"]
        tipo_usuario = data["tipo_usuario"]
        nombre = data["nombre"]
        apellido = data["apellido"]
        telefono = data.get("telefono")
        rfc = data.get("rfc")
        direccion = data.get("direccion")

        # Verificar si el usuario ya existe
        cursor.execute("SELECT COUNT(*) FROM usuarios WHERE usuario = :1", [usuario])
        if cursor.fetchone()[0] > 0:
            return jsonify({"error": "El nombre de usuario ya está en uso"}), 400

        # Verificar si el email ya existe
        cursor.execute("SELECT COUNT(*) FROM usuarios WHERE email = :1", [email])
        if cursor.fetchone()[0] > 0:
            return jsonify({"error": "El email ya está registrado"}), 400

        # Versión corregida del RETURNING INTO
        sql = """
            INSERT INTO usuarios (usuario, password, email, tipo_usuario)
            VALUES (:1, :2, :3, :4)
            RETURNING id_usuario INTO :5
        """
        id_usuario_var = cursor.var(oracledb.NUMBER)
        cursor.execute(sql, [usuario, password, email, tipo_usuario, id_usuario_var])

        id_usuario = id_usuario_var.getvalue()[0]  # Obtiene el primer valor del array
        print(id_usuario)

        if tipo_usuario == "CLIENTE":
            cursor.execute("""
                INSERT INTO clientes (id_usuario, nombre, apellido, telefono, rfc, direccion)
                VALUES (:1, :2, :3, :4, :5, :6)
            """, (id_usuario, nombre, apellido, telefono, rfc, direccion))

        conexion.commit()
        return jsonify({"mensaje": "Usuario registrado con éxito", "id_usuario": id_usuario}), 201

    except oracledb.DatabaseError as e:
        error_obj, = e.args
        conexion.rollback()
        if error_obj.code == 1:  # Código para ORA-00001
            return jsonify({"error": "El nombre de usuario o email ya está en uso"}), 400
        return jsonify({"error": str(e)}), 500

@app.route("/usuarios", methods=["GET"])
def obtenerUsuarios():
    cursor.execute("""
        SELECT * FROM usuarios
    """)

    columnas = [col[0].lower() for col in cursor.description]
    resultados = [dict(zip(columnas, fila)) for fila in cursor.fetchall()]
    return jsonify(resultados)


if __name__ == '__main__':
    app.run()
